# Importing libraries

In [1]:
#!/usr/bin/env python
%matplotlib inline

import time
import collections
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pyqtgraph as pg
from pyqtgraph.Qt import QtCore, QtGui
from scipy.signal import find_peaks
import seaborn as sns
import pickle

import sklearn
from sklearn import metrics
from sklearn import neighbors

import pandas as pd
import re
import csv
import os

import paho.mqtt.client as mqtt

In [2]:
import warnings
warnings.filterwarnings('ignore')

# Start MQTT Broker

In [ ]:
path = "C:\\Users\\ieliz\\Documents\\2021\\FYP\\Stress_Detection_FYP\\python_scripts\\"
broker_path = "C:\\Program Files (x86)\\Mosquitto"
os.chdir(broker_path)
print(os.getcwd())
os.system('cmd /k "mosquitto -c mosquitto_conf.conf -v"')

os.chdir(path)
print(os.getcwd())

C:\Program Files (x86)\Mosquitto


# Setting up MQTT connections

In [3]:
# declaring variables and callback functions
broker_address="192.168.1.125"

topic="data_reading"

def on_message(client, userdata, message):
    global rec_message
    rec_message = str(message.payload.decode("utf-8"))
    #print(rec_message)
    
# create client instance
# Client constructor: Client(client_id="", clean_session=True, userdata=None, protocol=MQTTv311, transport="tcp")
client = mqtt.Client("ComputerClient")
#print("Created client instance")

# when client receives message, it generates an on_message callback
client.on_message=on_message

# connecting to broker
client.connect(broker_address)
#print("Connected to broker")

0

# Setting up prediction

In [4]:
# Load the pickled model
saved_model_name = "knn_model_hrv_sc_1.sav"

clf = pickle.load(open(saved_model_name, 'rb'))
clf

KNeighborsClassifier(weights='distance')

In [5]:
def find_rmssd(data):
    rr_diff = np.diff(data)
    rr_diff[-1] = 0
    rr_sqdiff = np.power(rr_diff, 2)
    rmssd = np.sqrt(np.mean(rr_sqdiff))
    return rmssd

In [6]:
def predict_live(model, dataframe, window_size):
    
    # grab moving window of data
    size = len(dataframe.index)
    if size >= window_size:
        df = dataframe.iloc[-(window_size+1):].copy()
    else:
        df = dataframe.copy()
            
    # min-max norm
    min_val_gsr = min(df['Conductance (uS)'])
    max_val_gsr = max(df['Conductance (uS)'])
    scaling_gsr = max_val_gsr-min_val_gsr
    df.loc[:, 'Normalised_GSR'] = (df.loc[:, 'Conductance (uS)']- min_val_gsr)/scaling_gsr 
    
    min_val_hr = min(df["Heart_Rate"])
    max_val_hr = max(df["Heart_Rate"])
    scaling_hr = max_val_hr-min_val_hr
    df.loc[:, "Normalised_HR"] = (df.loc[:, "Heart_Rate"]- min_val_hr)/scaling_hr 
           
    HR_data = df["Heart_Rate"].to_numpy()
    GSR_data = df["Conductance (uS)"].to_numpy()
    HR_norm_data = df["Normalised_HR"].to_numpy()
    GSR_norm_data = df["Normalised_GSR"].to_numpy()
    HRV_data = df["HRV"].to_numpy()
                
    hr_mean = np.mean(HR_norm_data)       # hr mean
    hr_std = np.std(HR_norm_data)         # hr std
    hr_diff = max(HR_data) - min(HR_data) # hr diff
    
    hrv_mean = np.mean(HRV_data)          # hrv mean
    hrv_std = np.std(HRV_data)            # hrv std
    hrv_rmssd = find_rmssd(HRV_data)      # hrv rmssd
    
    gsr_mean = np.mean(GSR_norm_data)     # gsr mean
    gsr_std = np.std(GSR_norm_data)       # gsr std
        
    # find gsr peaks
    curr_peaks_ind, _ = find_peaks(GSR_norm_data)
    curr_peaks = GSR_norm_data[curr_peaks_ind]
    
    gsr_num_peaks = len(curr_peaks)       # gsr num peaks
    gsr_rate_peaks = gsr_num_peaks/len(GSR_norm_data)   # gsr rate peaks
    gsr_mean_peaks = np.mean(curr_peaks)  # gsr mean peaks
    gst_std_peaks = np.std(curr_peaks)    # gsr std peaks
            
    prediction = clf.predict(np.c_[hr_mean, hr_std, hr_diff, hrv_mean, hrv_std, hrv_rmssd, gsr_mean, gsr_std, gsr_rate_peaks, gsr_mean_peaks, gst_std_peaks])
        
    return prediction

# Reading in data

In [13]:
timeout = 5                       # Seconds
# filename = "assessment_test.csv"
tempname = "rory_lab_8_1"
max_num_readings = 60*60            # Seconds

r = re.compile("(?<==)([0-9]+)")

plotLength = 20

In [ ]:
client.loop_start()
print("Recording"+ " Started")

# run a loop otherwise, miss callbacks
client.loop_start()

# subscribing to topic
client.subscribe(topic)
print("Subscribed to topic", topic)

idx = 0
hr_data = []
hrv_data = []
gsr_data = []
resistance_data = []
conductance_data = []
prediction_data = []
readings_left = True
timeout_reached = False
rec_message = ""

df_predict = pd.DataFrame(columns = ['Time (hr:min:sec)', 'Time (s)', 'GSR', 'Resistance', 'Conductance (uS)', 'Heart_Rate', 'HRV', 'Section'])

filename = tempname + ".csv"

with open(filename, "w", newline='', encoding='utf-8') as f:
    writer = csv.writer(f, delimiter=',', quotechar='"')
    writer.writerow(['Time (hr:min:sec)', 'Time (s)', 'GSR', 'Resistance', 'Conductance (uS)', 'Heart_Rate', 'HRV', 'Section', 'Stressed'])
    
    while readings_left and not timeout_reached:
        global rec_message
        serial_line = rec_message
        time.sleep(1)
        try:
            idx += 1
            
            match = re.findall(r, serial_line)
            hr = int(match[0])
            gsr = int(match[1])
            batt = int(match[2])/100 # convert back to float
            #print(batt)
            #print(hr, gsr)

            hr_data.append(hr)
            
            hrv = 60000/hr
            hrv_data.append(hrv)

            resistance = (2**10 + 2*gsr)/(2**9-gsr)*10000
            gsr_data.append(gsr)
            resistance_data.append(resistance)

            conductance = 1/resistance*1000000
            conductance_data.append(conductance)

            if len(hr_data) >= max_num_readings:
                readings_left = False
                        
            t = time.strftime("%H:%M:%S", time.localtime())
            df_predict.loc[idx] = [t, idx, gsr, resistance, conductance, hr, hrv, tempname]
            
            if len(hr_data) >= 3:
                prediction = predict_live(clf, df_predict, 30)
                pred = prediction[0]
                prediction_data.append(pred)
                if pred >= 5:
                    print(batt, hr, gsr, pred, "Stressed")
                else:
                    print(batt, hr, gsr, pred, "Not Stressed")
                        
            writer.writerow([t, idx, gsr, resistance, conductance, hr, hrv, tempname, prediction_data[-1]])
            
        except:
            continue        

    print("Recording" + " Finished")
    
    # stop MQTT loop
    client.loop_stop()
    

Recording Started
Subscribed to topic data_reading


In [ ]:
client.loop_stop()

# Graphing data

In [ ]:
# First set up the figure, the axis, and the plot element we want to animate
fig, ax = plt.subplots()

times=np.arange(1, plotLength+1).tolist()
voltages = []

ax.set_xlim((0, plotLength))
ax.set_ylim((0, 4000))
ax.set_title('Feather Analog Read')
ax.set_xlabel("time (s)")
ax.set_ylabel("Voltage (mV)")

xs = []
ys = []

line, = ax.plot([], [], lw=2)

x_count = 0